In [12]:
import numpy as np
from scipy.optimize import minimize

# Function to load data
def load_data(file_path):
    data = np.genfromtxt(file_path, delimiter=',')
    X = data[:, :-1]
    y = data[:, -1]
    y = 2 * y - 1  # Convert labels to {1, -1}
    return X, y

# Define Gaussian kernel function
def gaussian_kernel(x1, x2, gamma):
    # Calculate the Gaussian kernel matrix
    n_samples_x1 = x1.shape[0]
    n_samples_x2 = x2.shape[0]
    kernel_matrix = np.zeros((n_samples_x1, n_samples_x2))
    for i in range(n_samples_x1):
        for j in range(n_samples_x2):
            kernel_matrix[i, j] = np.exp(-np.sum((x1[i] - x2[j]) ** 2) / (2 * gamma ** 2))
    return kernel_matrix

# Define the objective function for Gaussian SVM
def gaussian_objective_function(alpha, k, y):
    obj_function = -np.sum(alpha)
    alpha_y = np.multiply(alpha.reshape(-1, 1), y.reshape(-1, 1))
    obj_function += 0.5 * np.sum(np.multiply(np.matmul(alpha_y, alpha_y.T), k))
    return obj_function

# Implement Gaussian SVM training function
def gaussian_svm(C, gamma, X, y):
    num_samples = X.shape[0]
    
    # Define equality constraint for SVM
    cons = ({'type': 'eq', 'fun': lambda alpha: np.dot(alpha, y), 'jac': lambda alpha: y})
    
    # Compute the Gaussian kernel matrix
    kernel = gaussian_kernel(X, X, gamma)
    
    # Minimize the objective function subject to constraints
    alpha_optimized = minimize(lambda alpha: gaussian_objective_function(alpha, kernel, y), 
                               np.zeros(num_samples), 
                               method='SLSQP', 
                               bounds=[(0, C)] * num_samples, 
                               constraints=cons, 
                               options={'disp': False})
    
    return alpha_optimized.x

# Prediction function using trained model
def predict(X, alphas, X_train, y_train, gamma):
    predictions = []
    for sample in X:
        prediction = 0
        for i, alpha in enumerate(alphas):
            prediction += alpha * y_train[i] * np.exp(-gamma * np.linalg.norm(sample - X_train[i])**2)
        predictions.append(np.sign(prediction))
    return np.array(predictions)

# Calculate classification error
def classification_error(y_true, y_pred):
    return np.mean(y_true != y_pred)

# Load data
train_file_path = "C:\\Rushiil\\ML\\bank-note-1\\bank-note\\train.csv"
test_file_path = "C:\\Rushiil\\ML\\bank-note-1\\bank-note\\test.csv"

X_train, y_train = load_data(train_file_path)
X_test, y_test = load_data(test_file_path)

# Set values for gamma and C
gamma_values = [0.1, 0.5, 1, 5, 100]
C_values = [100/873, 500/873, 700/873]

best_gamma, best_C, best_error = None, None, float('inf')

# Test different combinations of gamma and C
for gamma in gamma_values:
    for C in C_values:
        alphas = gaussian_svm(C, gamma, X_train, y_train)
        
        # Make predictions using the trained alphas
        predictions_train = predict(X_train, alphas, X_train, y_train, gamma)
        predictions_test = predict(X_test, alphas, X_train, y_train, gamma)
        
        # Calculate errors
        train_error = classification_error(y_train, predictions_train)
        test_error = classification_error(y_test, predictions_test)
        
        print(f"Gamma={gamma}, C={C}: Train Error = {train_error}, Test Error = {test_error}")
        
        if test_error < best_error:
            best_error = test_error
            best_gamma = gamma
            best_C = C

print(f"\nBest combination is gamma={best_gamma}, C={best_C} with test error {best_error}")


Gamma=0.1, C=0.1145475372279496: Train Error = 0.016055045871559634, Test Error = 0.018
Gamma=0.1, C=0.572737686139748: Train Error = 0.016055045871559634, Test Error = 0.018
Gamma=0.1, C=0.8018327605956472: Train Error = 0.016055045871559634, Test Error = 0.018
Gamma=0.5, C=0.1145475372279496: Train Error = 0.008027522935779817, Test Error = 0.008
Gamma=0.5, C=0.572737686139748: Train Error = 0.008027522935779817, Test Error = 0.008
Gamma=0.5, C=0.8018327605956472: Train Error = 0.008027522935779817, Test Error = 0.008
Gamma=1, C=0.1145475372279496: Train Error = 0.0, Test Error = 0.002
Gamma=1, C=0.572737686139748: Train Error = 0.0, Test Error = 0.002
Gamma=1, C=0.8018327605956472: Train Error = 0.0, Test Error = 0.002
Gamma=5, C=0.1145475372279496: Train Error = 0.0, Test Error = 0.002
Gamma=5, C=0.572737686139748: Train Error = 0.0011467889908256881, Test Error = 0.0
Gamma=5, C=0.8018327605956472: Train Error = 0.0011467889908256881, Test Error = 0.002
Gamma=100, C=0.1145475372279